In [1]:
import random
n=10
V = [(random.uniform(5,75),random.uniform(5,55)) for i in range(n)]
import matplotlib.pyplot as plt
alpha=2

In [2]:
def t_cost(s,t): return ((s[0]-t[0])**2+(s[1]-t[1])**2)**(0.5)
def d_cost(s,t): return (1/alpha)*(((s[0]-t[0])**2+(s[1]-t[1])**2)**(0.5))

In [3]:
import itertools
def two_opt(ps):
    n = len(ps)
    res = list(range(n))
    update = True
    while update:
        update = False
        for (i,j) in itertools.combinations(range(n),2):
            if ((t_cost(ps[res[i]],ps[res[i+1]])+t_cost(ps[res[j]],ps[res[(j+1)%n]]))>
                (t_cost(ps[res[i]],ps[res[j]])+t_cost(ps[res[i+1]],ps[res[(j+1)%n]]))):
                res[i+1:j+1] = res[j:i:-1]
                update = True
                
    return res

In [4]:
import heapq
def mst(ps):
    n = len(ps)
    #prim
    mst = {v:[] for v in range(n)} # MST の隣接リスト
    X = set()
    heap = []
    for v in range(1,n):
        heapq.heappush(heap,(t_cost(ps[0],ps[v]),0,v))
    while len(heap)>0:
        (d,u,v)=heapq.heappop(heap)
        if v not in X:
            X.add(v)
            mst[u].append(v)
            mst[v].append(u)
            for w in range(n):
                if w not in X: heapq.heappush(heap,(t_cost(ps[v],ps[w]),v,w))
    #dfs
    res = []
    stack = [0]
    visited = [False]*n
    while len(stack)>0:
        v = stack.pop()
        if visited[v]: continue
        visited[v]=True
        res.append(v)
        for u in mst[v]: stack.append(u)
    return res 

In [5]:
def dp(ps):
    length = {} # length[(u,S)]: u を始点とし S の点すべてを回る最小経路長、ディクショナリ
    route = {} # route[(u,S)]: 最小経路長を達成するためのルート、ディクショナリ
    v = ps[0]
    n = len(ps)
    for i in range(1,n+1):
        for a in itertools.combinations(range(n),i):
            S = frozenset(a)
            for j in S:
                u = ps[j]
                if i==1:
                    length[(j,S)] = t_cost(v,u)
                    route[(j,S)] = [j]
                else:
                    Sj = S-set([j]) # S から j を除いたもの
                    k=min(Sj,key=lambda k: length[(k,Sj)]+t_cost(ps[k],u))#Sjの中でlength[(k,Sj)]+dist(ps[k],u)が最小のものを返している
                    length[(j,S)] = length[(k,Sj)]+t_cost(ps[k],u)
                    route[(j,S)] = route[(k,Sj)]+[j]
    return route[(0,frozenset(range(n)))] 

In [1]:
def t_pathcost(a,b,tsp):
    pathcost=0
    if a<b:
        for i in range(a,b):
            pathcost=pathcost+t_cost(V[tsp[i]],V[tsp[(i+1)%len(tsp)]])
    if a>b:
        for i in range(a,len(tsp)):
            pathcost=pathcost+t_cost(V[tsp[i]],V[tsp[(i+1)%len(tsp)]])
        for i in range(b):
            pathcost=pathcost+t_cost(V[tsp[i]],V[tsp[(i+1)%len(tsp)]])
    if a==b:
        pathcost=0
            
    return pathcost

In [128]:
import itertools
import numpy
def greedy_core(tsp,depot):#depotはindex指定ではない
    label={}
    operation={}
    flyop={}
    driveop={}
    tsptotal=0
    for i in tsp:
        label[i]='simple'
        tsptotal=tsptotal+t_cost(V[tsp[i]],V[tsp[(i+1)%len(tsp)]])
    n=len(label)
    tspdtotal=tsptotal
    while 'simple' in label.values():
        ms=[]
        for i in range(n):
            if label[tsp[i]]!='simple' or tsp[i]==depot:
                ms.append(-float('inf'))
            else:
                mfsi=t_cost(V[tsp[i-1]],V[tsp[i]])+t_cost(V[tsp[i]],V[tsp[(i+1)%n]])-max(d_cost(V[tsp[i-1]],V[tsp[i]])+d_cost(V[tsp[i]],V[tsp[(i+1)%n]]),t_cost(V[tsp[i-1]],V[tsp[(i+1)%n]]))
                ms.append(mfsi)
        for i in range(n):
            if label[tsp[i-1]]=='combined' and tsp[(i-1)]!=depot:
                if label[tsp[i]]=='simple':
                    j=i-2
                    while label[tsp[j]]!='combined':
                        if label[tsp[j]]=='drone':
                            dronenode=j
                        if label[tsp[(j-1)]]=='combined':
                            pls=t_cost(V[tsp[i-1]],V[tsp[i]])+operation[(tsp[dronenode],tsp[j-1],tsp[i-1])]-max(d_cost(V[tsp[j-1]],V[tsp[dronenode]])+d_cost(V[tsp[dronenode]],V[tsp[i]]),driveop[(tsp[dronenode],tsp[j-1],tsp[i-1])]+t_cost(V[tsp[i-1]],V[tsp[i]]))
                            ms.append(pls)
                            break
                        j=j-1  
                else: ms.append(-float('inf'))
            else: ms.append(-float('inf'))
        for i in range(n):
            if label[tsp[(i+1)%n]]=='combined' and tsp[(i+1)%n]!=depot:
                if label[tsp[i]]=='simple':
                    j=(i+2)%n
                    while label[tsp[j]]!='combined':
                        if label[tsp[j]]=='drone':
                            dronenode=j
                        if label[tsp[(j+1)%n]]=='combined':
                            prs=t_cost(V[tsp[i]],V[tsp[(i+1)%n]])+operation[(tsp[dronenode],tsp[(i+1)%n],tsp[(j+1)%n])]-max(d_cost(V[tsp[i]],V[tsp[dronenode]])+d_cost(V[tsp[dronenode]],V[tsp[(j+1)%n]]),driveop[(tsp[dronenode],tsp[(i+1)%n],tsp[(j+1)%n])]+t_cost(V[tsp[i]],V[tsp[(i+1)%n]]))
                            ms.append(prs)
                            break
                        j=(j+1)%n
                else: ms.append(-float('inf'))
            else: ms.append(-float('inf'))
        i = numpy.argmax(ms)
        if ms[i]>0:
            tspdtotal=tspdtotal-ms[i]
            if i//n==0:#makeflysaving
                label[tsp[i]]='drone'
                label[tsp[i-1]]='combined'
                label[tsp[(i+1)%n]]='combined'
                flyop[(tsp[i],tsp[i-1],tsp[(i+1)%n])]=d_cost(V[tsp[i-1]],V[tsp[i]])+d_cost(V[tsp[i]],V[tsp[(i+1)%n]])
                driveop[(tsp[i],tsp[i-1],tsp[(i+1)%n])]=t_cost(V[tsp[i-1]],V[tsp[(i+1)%n]])
                operation[(tsp[i],tsp[i-1],tsp[(i+1)%n])]=max(flyop[(tsp[i],tsp[i-1],tsp[(i+1)%n])],driveop[(tsp[i],tsp[i-1],tsp[(i+1)%n])])
            elif i//n==1:#pushleftsaving
                label[tsp[i%n]]='combined'
                label[tsp[(i-1)%n]]='truck'
                j=(i-2)%n
                while label[tsp[j]]!='combined':
                    if label[tsp[j]]=='drone':
                        dronenode=j
                    if label[tsp[j-1]]=='combined':
                        flyop[(tsp[dronenode],tsp[j-1],tsp[i%n])]=d_cost(V[tsp[j-1]],V[tsp[dronenode]])+d_cost(V[tsp[dronenode]],V[tsp[i%n]])
                        driveop[(tsp[dronenode],tsp[j-1],tsp[i%n])]=driveop[(tsp[dronenode],tsp[j-1],tsp[(i-1)%n])]+t_cost(V[tsp[(i-1)%n]],V[tsp[i%n]])
                        operation[(tsp[dronenode],tsp[j-1],tsp[i%n])]=max(flyop[(tsp[dronenode],tsp[j-1],tsp[i%n])],driveop[(tsp[dronenode],tsp[j-1],tsp[i%n])])
                        break
                    j=j-1
            elif i//n==2:#pushrightsaving
                label[tsp[i%n]]='combined'
                label[tsp[(i+1)%n]]='truck'
                j=(i+2)%n
                while label[tsp[j]]!='combined':
                    if label[tsp[j]]=='drone':
                        dronenode=j
                    if label[tsp[(j+1)%n]]=='combined':
                        flyop[(tsp[dronenode],tsp[i%n],tsp[(j+1)%n])]=d_cost(V[tsp[i%n]],V[tsp[dronenode]])+d_cost(V[tsp[dronenode]],V[tsp[(j+1)%n]])
                        driveop[(tsp[dronenode],tsp[i%n],tsp[(j+1)%n])]=driveop[(tsp[dronenode],tsp[(i+1)%n],tsp[(j+1)%n])]+t_cost(V[tsp[i%n]],V[tsp[(i+1)%n]])
                        operation[(tsp[dronenode],tsp[i%n],tsp[(j+1)%n])]=max(flyop[(tsp[dronenode],tsp[i%n],tsp[(j+1)%n])],driveop[(tsp[dronenode],tsp[i%n],tsp[(j+1)%n])])
                        break
                    j=(j+1)%n
        else:
            for l in label:
                if label[l]=='simple':
                    label[l]='combined'

      
    return tspdtotal,label
            

In [89]:
def greedy_ph(alg,V,depot):
    tsp=alg(V)
    return greedy_core(tsp,depot)

In [275]:
import itertools
def exact_core(tsp,depot):
    T={}
    M={}
    D={}
    Dres=0
    Mdrone={}
    route={}
    n=len(tsp)
    for (start,drone,end) in itertools.combinations(range(n),3):
        drone_cost=d_cost(V[tsp[start]],V[tsp[drone]])+d_cost(V[tsp[drone]],V[tsp[end]])
        truck_cost=t_pathcost(start,drone-1,tsp)+t_cost(V[tsp[drone-1]],V[tsp[drone+1]])+t_pathcost(drone+1,end,tsp)
        T[start,drone,end]=max(drone_cost,truck_cost)
    for (end,start,drone) in itertools.combinations(range(n),3):
        drone_cost=d_cost(V[tsp[start]],V[tsp[drone]])+d_cost(V[tsp[drone]],V[tsp[end]])
        truck_cost=t_pathcost(start,drone-1,tsp)+t_cost(V[tsp[drone-1]],V[tsp[(drone+1)%n]])+t_pathcost((drone+1)%n,end,tsp)
        T[start,drone,end]=max(drone_cost,truck_cost)
    for (drone,end,start) in itertools.combinations(range(n),3):
        drone_cost=d_cost(V[tsp[start]],V[tsp[drone]])+d_cost(V[tsp[drone]],V[tsp[end]])
        truck_cost=t_pathcost(start,drone-1,tsp)+t_cost(V[tsp[drone-1]],V[tsp[(drone+1)%n]])+t_pathcost((drone+1)%n,end,tsp)
        T[start,drone,end]=max(drone_cost,truck_cost)
        
    for (i,j) in itertools.permutations(range(n),2):
        M[i,j]=float('inf')
        if i<j:
            if i+1==j:
                M[i,j]=t_cost(V[tsp[i]],V[tsp[j]])
            for k in range(i+1,j):
                if M[i,j]>T[i,k,j]:
                    M[i,j]=T[i,k,j]
                    Mdrone[tsp[i],tsp[j]]=[tsp[k]]
        else:
            for k in range(i+1,n):
                if M[i,j]>T[i,k,j]:
                    M[i,j]=T[i,k,j]
                    Mdrone[tsp[i],tsp[j]]=[tsp[k]]
            for k in range(0,j):
                if M[i,j]>T[i,k,j]:
                    M[i,j]=T[i,k,j]
                    Mdrone[tsp[i],tsp[j]]=[tsp[k]]
            M[n-1,0]=t_cost(V[tsp[n-1]],V[tsp[0]])

    for i in range(tsp.index(depot)+1,tsp.index(depot)+n+1):
        D[depot,tsp[i%n]]=float('inf')
        Dres=float('inf')
        D[depot,depot]=0
        route[depot,depot]=[[depot]]
        for j in range(tsp.index(depot),i):
            if j%n!=i%n:
                d=M[j%n,i%n]+D[depot,tsp[j%n]]
                r=route[depot,tsp[j%n]]+[[tsp[j%n],tsp[i%n]]]
                if i==tsp.index(depot)+n:
                    if Dres>d:
                        Dres=d
                        route[depot,depot]=r

                else:
                    if D[depot,tsp[i%n]]>d:
                        D[depot,tsp[i%n]]=d
                        route[depot,tsp[i%n]]=r


    pathlen=len(route[depot,depot])
    path=[]
    for i in range(pathlen-1):
        path=path+route[depot,depot][i+1]
        if (route[depot,depot][i+1][0],route[depot,depot][i+1][1]) in Mdrone:
            path=path+[Mdrone[route[depot,depot][i+1][0],route[depot,depot][i+1][1]]]
        else:
            path=path+[[]]
    
        
    
    return Dres,path

In [34]:
import itertools
def exact_ph(alg,V,depot):
    tsp=alg(V)
    return exact_core(tsp,depot)

In [35]:
import itertools
def twopmove(lis,i,j):#iとjの順番を入れ替える
    if i<j: new_list=lis[:i]+[lis[j]]+lis[(i+1):j]+[lis[i]]+lis[(j+1):]
    else: new_list=lis[:j]+[lis[i]]+lis[(j+1):i]+[lis[j]]+lis[(i+1):]
    return new_list
         
def twooptmove(lis,i,j):#2辺を入れ替える
    if i<j: new_list=lis[:i+1]+lis[j:(i+1)%len(lis)-1:-1]+lis[(j+1):]
    else: new_list=[lis[i]]+lis[j::-1]+lis[:(i+1)%len(lis)-1:-1]+lis[j+1:i]
        
    return new_list

def onepmove(lis,i,j):#iをj番目に挿入する
    if i<j: new_list=lis[:i]+lis[(i+1):(j+1)]+[lis[i]]+lis[(j+1):]
    else: new_list=lis[:j]+[lis[i]]+lis[j:i]+lis[(i+1):]

    return new_list
    

In [133]:
def improve(move,heuristic,lis,depot):
    n=len(lis)
    initial=lis
    f=heuristic(initial,depot)[0]
    update=True
    while update:
        update=False
        for (i,j) in itertools.permutations(range(n),2):
            newroute=move(lis,i,j)
            if f>heuristic(newroute,depot)[0]:
                update=True
                initial=newroute
                f=heuristic(newroute,depot)[0]
        if update:
            lis=initial
    return initial

def improve_all(heuristic,lis,depot):                                
    n=len(lis)
    initial=lis
    f=heuristic(initial,depot)[0]
    update=True
    while update:
        update=False
        for (i,j) in itertools.permutations(range(n),2):
            moves=[twopmove(lis,i,j),twooptmove(lis,i,j),onepmove(lis,i,j)]
            for newroute in moves:
                if f>heuristic(newroute,depot)[0]:
                    update=True
                    initial=newroute
                    f=heuristic(newroute,depot)[0]
        if update:
            lis=initial
    return initial
    
    


In [13]:
def improve_twopmove_greedy(alg,V,depot):
    tsp=improve(twopmove,greedy_core,alg(V),depot)
    return greedy_core(tsp,depot)

def improve_twooptmove_greedy(alg,V,depot):
    tsp=improve(twooptmove,greedy_core,alg(V),depot)
    return greedy_core(tsp,depot)

def improve_onepmove_greedy(alg,V,depot):
    tsp=improve(onepmove,greedy_core,alg(V),depot)
    return greedy_core(tsp,depot)

def improve_twopmove_exact(alg,V,depot):
    tsp=improve(twopmove,exact_core,alg(V),depot)
    return exact_core(tsp,depot)

def improve_twooptmove_exact(alg,V,depot):
    tsp=improve(twooptmove,exact_core,alg(V),depot)
    return exact_core(tsp,depot)

def improve_onepmove_exact(alg,V,depot):
    tsp=improve(onepmove,exact_core,alg(V),depot)
    return exact_core(tsp,depot)

In [14]:
def improve_all_greedy(alg,V,depot):
    tsp=improve_all(greedy_core,alg(V),depot)
    return greedy_core(tsp,depot)

def improve_all_exact(alg,V,depot):
    tsp=improve_all(exact_core,alg(V),depot)
    return exact_core(tsp,depot)